# Masakhane - Machine Translation for African Languages (Using JoeyNMT)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "xh" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead! 
!mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
g_drive_path = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)
os.environ["gdrive_path"] = g_drive_path
models_path = '%s/models/%s%s_transformer'% (g_drive_path, source_language, target_language)
# model temporary directory for training
model_temp_dir = "/content/drive/My Drive/masakhane/model-temp"

In [3]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/en-xh-baseline


In [4]:
#TODO: Need not run for for retrain
# Install opus-tools
! pip install opustools-pkg 

     |████████████████████████████████| 81kB 11.1MB/s 


In [5]:
#TODO: Need not run for for retrain
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-xh.xml.gz not found. The following files are available for downloading:

   7 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en-xh.xml.gz
 263 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en.zip
  78 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/xh.zip

 348 MB Total size
./JW300_latest_xml_en-xh.xml.gz ... 100% of 7 MB
./JW300_latest_xml_en.zip ... 100% of 263 MB
./JW300_latest_xml_xh.zip ... 100% of 78 MB


In [6]:
#TODO: Need not run for for retrain
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2020-01-31 11:19:59--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.01s   

2020-01-31 11:19:59 (23.4 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2020-01-31 11:20:04--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-xh.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

In [7]:
#TODO: Need not run for for retrain
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [8]:
#TODO: Need not run for for retrain
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 7158/876188 lines since contained in test set.


,source_sentence,target_sentence
0,How One Marriage Was Saved,Indlela Owasindiswa Ngayo Lo Mtshato
1,“ The application of the counsel in the book M...,Omnye umfundi onoxabiso nowaseMzantsi Afrika w...
2,"“ Chapter 5 , ‘ A Wife Who Is Dearly Loved , ’...","“ Isahluko 5 , esithi ‘ Umfazi Othandwa Kunene..."


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [9]:
#TODO: Need not run for for retrain
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [10]:
#TODO: Need not run for for retrain
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np

# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

# NOTE - This might run slow depending on the size of your training set. We are
# printing some information to help you track how long it would take. 
scores = []
start_time = time.time()
for idx, row in df_pp.iterrows():
  scores.append(fuzzfilter(row['source_sentence'], list(en_test_sents), 5))
  if idx % 1000 == 0:
    hours, rem = divmod(time.time() - start_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds), "%0.2f percent complete" % (100.0*float(idx)/float(len(df_pp))))

# Filter out "almost overlapping samples"
df_pp['scores'] = scores
df_pp = df_pp[df_pp['scores'] < 95]

     |████████████████████████████████| 51kB 6.0MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144675 sha256=c603320b54916b69dc5d26f92c63f9a5b5df8a2a0c4926898fe11de36cad266c
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein
00:00:00.26 0.00 percent complete
00:00:18.30 0.13 percent complete
00:00:36.75 0.25 percent complete
00:00:55.49 0.38 percent complete
00:01:13.98 0.51 percent complete
00:01:32.77 0.63 percent complete
00:01:50.78 0.76 percent complete
00:02:08.95 0.88 percent complete
00:02:27.15 1.01 percent complete
00:02:45.84 1.14 percent complete
00:03:04.76 1.26 percent complete
00:03:23.44 1.39 percent complete
00:03:41.86 1.52 percent complete
00:04:00.52 1.64 percent complete
00:04:18.59 1.77 percent complete
00:04:36.64 1.89 percent complete
00:04:55.39 2.02 percent complete
00:05:13.84 2.15 percent complete
00

00:15:05.54 6.19 percent complete
00:15:23.96 6.31 percent complete
00:15:41.57 6.44 percent complete
00:16:00.46 6.57 percent complete
00:16:18.03 6.69 percent complete


00:16:35.78 6.82 percent complete
00:16:54.71 6.94 percent complete
00:17:12.70 7.07 percent complete
00:17:30.78 7.20 percent complete
00:17:49.67 7.32 percent complete
00:18:08.25 7.45 percent complete
00:18:26.55 7.58 percent complete
00:18:45.16 7.70 percent complete
00:19:04.12 7.83 percent complete
00:19:22.51 7.95 percent complete
00:19:40.97 8.08 percent complete
00:19:59.66 8.21 percent complete
00:20:18.00 8.33 percent complete


00:20:35.81 8.46 percent complete
00:20:54.57 8.59 percent complete
00:21:12.73 8.71 percent complete
00:21:30.86 8.84 percent complete
00:21:49.29 8.96 percent complete
00:22:07.06 9.09 percent complete


00:22:25.55 9.22 percent complete
00:22:43.84 9.34 percent complete
00:23:02.31 9.47 percent complete
00:23:20.83 9.60 percent complete
00:23:39.24 9.72 percent complete
00:23:57.72 9.85 percent complete
00:24:16.26 9.97 percent complete
00:24:33.77 10.10 percent complete
00:24:52.62 10.23 percent complete
00:25:10.83 10.35 percent complete
00:25:28.97 10.48 percent complete
00:25:47.46 10.61 percent complete
00:26:06.37 10.73 percent complete
00:26:24.78 10.86 percent complete
00:26:43.19 10.98 percent complete
00:27:01.43 11.11 percent complete
00:27:19.17 11.24 percent complete
00:27:37.29 11.36 percent complete
00:27:55.97 11.49 percent complete
00:28:14.30 11.62 percent complete
00:28:33.50 11.74 percent complete
00:28:52.37 11.87 percent complete
00:29:10.65 11.99 percent complete
00:29:28.52 12.12 percent complete
00:29:47.24 12.25 percent complete
00:30:05.14 12.37 percent complete
00:30:23.28 12.50 percent complete
00:30:41.87 12.63 percent complete
00:31:00.26 12.75 percent c

...

03:28:27.87 86.23 percent complete
03:28:45.90 86.36 percent complete
03:29:04.25 86.48 percent complete
03:29:22.24 86.61 percent complete
03:29:40.38 86.73 percent complete
03:29:58.83 86.86 percent complete
03:30:16.88 86.99 percent complete
03:30:35.07 87.11 percent complete
03:30:54.09 87.24 percent complete
03:31:11.99 87.37 percent complete
03:31:29.91 87.49 percent complete
03:31:47.80 87.62 percent complete
03:32:05.70 87.74 percent complete
03:32:23.91 87.87 percent complete
03:32:42.28 88.00 percent complete
03:33:00.62 88.12 percent complete
03:33:18.49 88.25 percent complete
03:33:36.64 88.38 percent complete
03:33:54.61 88.50 percent complete
03:34:12.63 88.63 percent complete
03:34:30.96 88.75 percent complete
03:34:49.25 88.88 percent complete
03:35:07.86 89.01 percent complete
03:35:26.65 89.13 percent complete
03:35:45.36 89.26 percent complete
03:36:03.59 89.39 percent complete
03:36:21.08 89.51 percent complete
03:36:39.09 89.64 percent complete
03:36:57.04 89.76 pe

03:39:20.52 90.77 percent complete
03:39:38.44 90.90 percent complete
03:39:56.86 91.03 percent complete
03:40:15.22 91.15 percent complete
03:40:33.33 91.28 percent complete
03:40:51.69 91.41 percent complete
03:41:09.49 91.53 percent complete
03:41:27.66 91.66 percent complete
03:41:46.10 91.78 percent complete
03:42:04.25 91.91 percent complete
03:42:22.88 92.04 percent complete
03:42:41.57 92.16 percent complete
03:42:59.68 92.29 percent complete
03:43:17.98 92.42 percent complete
03:43:36.06 92.54 percent complete
03:43:54.54 92.67 percent complete
03:44:13.04 92.79 percent complete
03:44:31.23 92.92 percent complete
03:44:50.10 93.05 percent complete
03:45:08.41 93.17 percent complete
03:45:27.13 93.30 percent complete


03:45:45.34 93.43 percent complete
03:46:04.25 93.55 percent complete
03:46:22.11 93.68 percent complete
03:46:39.82 93.80 percent complete
03:46:58.31 93.93 percent complete
03:47:16.92 94.06 percent complete
03:47:34.82 94.18 percent complete
03:47:53.48 94.31 percent complete
03:48:11.41 94.44 percent complete
03:48:29.59 94.56 percent complete
03:48:47.86 94.69 percent complete
03:49:05.70 94.81 percent complete
03:49:24.02 94.94 percent complete
03:49:42.04 95.07 percent complete
03:50:00.65 95.19 percent complete
03:50:18.86 95.32 percent complete
03:50:37.31 95.45 percent complete
03:50:55.82 95.57 percent complete
03:51:13.92 95.70 percent complete
03:51:32.28 95.82 percent complete
03:51:50.32 95.95 percent complete
03:52:08.59 96.08 percent complete
03:52:26.93 96.20 percent complete
03:52:45.17 96.33 percent complete
03:53:03.67 96.46 percent complete
03:53:22.27 96.58 percent complete
03:53:40.63 96.71 percent complete
03:53:59.02 96.83 percent complete
03:54:17.40 96.96 pe

03:56:05.88 97.72 percent complete
03:56:23.84 97.84 percent complete
03:56:42.22 97.97 percent complete
03:57:00.68 98.10 percent complete
03:57:18.45 98.22 percent complete
03:57:36.79 98.35 percent complete
03:57:55.08 98.48 percent complete
03:58:13.94 98.60 percent complete
03:58:32.58 98.73 percent complete
03:58:50.85 98.85 percent complete


03:59:09.34 98.98 percent complete
03:59:28.37 99.11 percent complete
03:59:46.59 99.23 percent complete
04:00:04.88 99.36 percent complete
04:00:22.93 99.49 percent complete
04:00:41.14 99.61 percent complete
04:00:59.34 99.74 percent complete
04:01:17.44 99.86 percent complete
04:01:35.66 99.99 percent complete


In [11]:
#TODO: Need not run for for retrain
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.en <==
Not long after my baptism , I married one of Jehovah’s Witnesses .
Yet , he could not take sole blame for the horrors to come .
How do we gain the help of holy spirit ?
Faith moved a prostitute to risk her life so as to protect two Israelite spies .
At some points , this wall stood 165 feet ( 50 m ) high .
He loved me dearly , something I failed to realize at the time .
His sister , Christine Antisdel , who had recently become one of Jehovah’s Witnesses , offered to move in with him and help him care for his two very young children .
However , they should contain not only requests but also expressions of faith .
At that point , any interest I had in winning the approval of my schoolmates would fade away .
Jesus ’ brothers now urge him : “ Pass on over from here and go into Judea . ”

==> train.xh <==
Kungekudala ndibhaptiziwe , ndatshata omnye wamaNgqina kaYehova .
Kodwa ke , yayingenguye yedwa owabangela iziphendu ezaziza kulandela .
Silufumana njani uncedo lomoya oyi



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [12]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 2256 (delta 43), reused 32 (delta 18), pack-reused 2184
Receiving objects: 100% (2256/2256), 2.63 MiB | 1.94 MiB/s, done.
Resolving deltas: 100% (1564/1564), done.
Processing /content/joeynmt
     |████████████████████████████████| 276kB 40.8MB/s 
     |████████████████████████████████| 307kB 49.1MB/s 
     |████████████████████████████████| 51kB 9.0MB/s 
     |████████████████████████████████| 215kB 56.2MB/s 
     |████████████████████████████████| 747kB 58kB/s 
     |████████████████████████████████| 61kB 10.1MB/s 
  Created wheel for joeynmt: filename=joeynmt-0.0.1-cp36-none-any.whl size=72567 sha256=5f622310d02f497ec2997f69ee681768c4e004faad455eeb405c09f5f9b2f82b
  Stored in directory: /tmp/pip-ephem-wheel-cache-k6vjzzhq/wheels/db/01/db/751cc9f3e7f6faec127c43644ba250a3ea7ad200594aeda70a
  Created wheel for pyya

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [25]:
#TODO: Need not run for for retrain
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path "$gdrive_path/vocab.txt"

# Some output
! echo "BPE Xhosa Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 "$gdrive_path/vocab.txt"  # Herman

bpe.codes.4000	dev.en	     test.bpe.xh     test.xh	   train.en
dev.bpe.en	dev.xh	     test.en	     train.bpe.en  train.xh
dev.bpe.xh	test.bpe.en  test.en-any.en  train.bpe.xh  vocab.txt
bpe.codes.4000	dev.en	     test.bpe.xh     test.xh	   train.en
dev.bpe.en	dev.xh	     test.en	     train.bpe.en  train.xh
dev.bpe.xh	test.bpe.en  test.en-any.en  train.bpe.xh
BPE Xhosa Sentences
Oku kwa@@ ph@@ umela ekubeni nd@@ id@@ ume njengom@@ ntu ong@@ any@@ anis@@ ekanga .
Xa nd@@ afunda inyaniso , and@@ izange nd@@ iph@@ inde nd@@ iv@@ ume ukuq@@ hubeka ndis@@ enza oko , naku@@ beni ndand@@ ihl@@ awul@@ wa um@@ v@@ uzo on@@ c@@ um@@ isayo .
Nd@@ ing@@ umzekelo om@@ hle ko@@ onyana bam abab@@ ini yaye nd@@ iye nd@@ af@@ anel@@ ekela amalung@@ elo ang@@ akumbi ebandleni .
Nd@@ az@@ iwa njengom@@ ntu on@@ yanisekileyo ngabantu end@@ ish@@ ish@@ ina kunye nabo kwan@@ abahl@@ ol@@ i boku@@ m@@ a kwem@@ ali ye@@ enk@@ amp@@ ani ( t@@ a@@ x a@@ ud@@ it@@ ors ) . ”
U@@ R@@ ute wa@@ f@@ ud@@ ukela kwa@@ S

# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [0]:
def get_last_checkpoint(directory):
  last_checkpoint = ''
  try:
    for filename in os.listdir(directory):
      if not 'best' in filename and filename.endswith(".ckpt"):
          if not last_checkpoint or int(filename.split('.')[0]) > int(last_checkpoint.split('.')[0]):
            last_checkpoint = filename
  except FileNotFoundError as e:
    print('Error Occur ', e)
  return last_checkpoint

In [28]:
# Copy the created models from the temporary storage to main storage on google drive for persistant storage 
# the content of te folder will be overwrite when you start trainin
!cp -r "/content/drive/My Drive/masakhane/model-temp/"* "$gdrive_path/models/${src}${tgt}_transformer/"
last_checkpoint = get_last_checkpoint(models_path)
print('Last checkpoint :',last_checkpoint)

cp: cannot stat '/content/drive/My Drive/masakhane/model-temp/*': No such file or directory
Error Occur  [Errno 2] No such file or directory: '/content/drive/My Drive/masakhane/en-xh-baseline/models/enxh_transformer'
Last checkpoint : 


In [0]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{gdrive_path}/train.bpe"
    dev:   "{gdrive_path}/dev.bpe"
    test:  "{gdrive_path}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "{gdrive_path}/vocab.txt"
    trg_vocab: "{gdrive_path}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/{last_checkpoint}" # uncommented to load a pre-trained model from last checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "{model_temp_dir}"
    overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language, model_temp_dir=model_temp_dir, last_checkpoint=last_checkpoint)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [0]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

2020-01-31 18:29:28,118 Hello! This is Joey-NMT.
2020-01-31 18:29:29,295 Total params: 12200448
2020-01-31 18:29:29,296 Trainable parameters: ['decoder.layer_norm.bias', 'decoder.layer_norm.weight', 'decoder.layers.0.dec_layer_norm.bias', 'decoder.layers.0.dec_layer_norm.weight', 'decoder.layers.0.feed_forward.layer_norm.bias', 'decoder.layers.0.feed_forward.layer_norm.weight', 'decoder.layers.0.feed_forward.pwff_layer.0.bias', 'decoder.layers.0.feed_forward.pwff_layer.0.weight', 'decoder.layers.0.feed_forward.pwff_layer.3.bias', 'decoder.layers.0.feed_forward.pwff_layer.3.weight', 'decoder.layers.0.src_trg_att.k_layer.bias', 'decoder.layers.0.src_trg_att.k_layer.weight', 'decoder.layers.0.src_trg_att.output_layer.bias', 'decoder.layers.0.src_trg_att.output_layer.weight', 'decoder.layers.0.src_trg_att.q_layer.bias', 'decoder.layers.0.src_trg_att.q_layer.weight', 'decoder.layers.0.src_trg_att.v_layer.bias', 'decoder.layers.0.src_trg_att.v_layer.weight', 'decoder.layers.0.trg_trg_att.k_l

In [22]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!cp -r joeynmt/models/${src}${tgt}_transformer/* "$gdrive_path/models/${src}${tgt}_transformer/"

cp: cannot stat 'joeynmt/models/enxh_transformer/*': No such file or directory


In [23]:
# Output our validation accuracy
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

cat: '/content/drive/My Drive/masakhane/en-xh-baseline/models/enxh_transformer/validations.txt': No such file or directory


In [24]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"

2020-01-31 18:12:49,600 Hello! This is Joey-NMT.
Traceback (most recent call last):
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/content/joeynmt/joeynmt/__main__.py", line 41, in <module>
    main()
  File "/content/joeynmt/joeynmt/__main__.py", line 32, in main
    output_path=args.output_path, save_attention=args.save_attention)
  File "/content/joeynmt/joeynmt/prediction.py", line 187, in test
    cfg = load_config(cfg_file)
  File "/content/joeynmt/joeynmt/helpers.py", line 164, in load_config
    with open(path, 'r') as ymlfile:
FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/masakhane/en-xh-baseline/models/enxh_transformer/config.yaml'
